# Pickling cuML Models for Persistence

This notebook demonstrates simple pickling of both single-GPU and multi-GPU cuML models for persistence

In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

## Single GPU Model Pickling

All single-GPU estimators are pickleable. The following example demonstrates the creation of a synthetic dataset, training, and pickling of the resulting model for storage. Trained single-GPU models can also be used to distribute the inference on a Dask cluster, which the `Distributed Model Pickling` section below demonstrates.

In [2]:
from cuml.datasets import make_blobs

X, y = make_blobs(n_samples=50,
                  n_features=10,
                  centers=5,
                  cluster_std=0.4,
                  random_state=0)

In [3]:
from cuml.cluster import KMeans

model = KMeans(n_clusters=5)

model.fit(X)

KMeans()

In [4]:
import pickle

pickle.dump(model, open("kmeans_model.pkl", "wb"))

In [5]:
model = pickle.load(open("kmeans_model.pkl", "rb"))

In [6]:
model.cluster_centers_

array([[-5.7684636 ,  2.3276033 , -3.7457774 , -1.8541754 , -5.1695833 ,
         7.667088  ,  2.7118318 ,  8.495609  ,  1.7038484 ,  1.1884269 ],
       [ 4.647688  ,  8.37788   , -9.070581  ,  9.459332  ,  8.450423  ,
        -1.0210547 ,  3.3920872 , -7.862985  , -0.75276613,  0.48384127],
       [-2.9414442 ,  4.6401706 , -4.5027533 ,  2.2855108 ,  1.644645  ,
        -2.4937892 , -5.2241607 , -1.5499196 , -8.063638  ,  2.816936  ],
       [-4.271077  ,  5.5611653 , -5.6640916 , -1.8229512 , -9.2925    ,
         0.730283  ,  4.4586773 , -2.8876226 , -5.1257744 ,  9.694357  ],
       [ 5.5837426 , -4.1515303 ,  4.369667  , -3.00205   ,  3.638897  ,
        -4.3419113 , -3.318711  ,  6.503671  , -6.865036  , -1.0266498 ]],
      dtype=float32)

## Distributed Model Pickling

The distributed estimator wrappers inside of the `cuml.dask` are not intended to be pickled directly. The Dask cuML estimators provide a function `get_combined_model()`, which returns the trained single-GPU model for pickling. The combined model can be used for inference on a single-GPU, and the `ParallelPostFit` wrapper from the [Dask-ML](https://ml.dask.org/meta-estimators.html) library can be used to perform distributed inference on a Dask cluster.

In [7]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster

cluster = LocalCUDACluster()
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:35546 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 1 Cores: 1 Memory: 251.80 GiB


In [8]:
from cuml.dask.datasets import make_blobs

n_workers = len(client.scheduler_info()["workers"].keys())

X, y = make_blobs(n_samples=5000, 
                  n_features=30,
                  centers=5, 
                  cluster_std=0.4, 
                  random_state=0,
                  n_parts=n_workers*5)

X = X.persist()
y = y.persist()

In [9]:
from cuml.dask.cluster import KMeans

dist_model = KMeans(n_clusters=5)

In [10]:
dist_model.fit(X)

In [11]:
import pickle

single_gpu_model = dist_model.get_combined_model()
pickle.dump(single_gpu_model, open("kmeans_model.pkl", "wb"))

In [12]:
single_gpu_model = pickle.load(open("kmeans_model.pkl", "rb"))

In [13]:
single_gpu_model.cluster_centers_

array([[ 4.809872  ,  8.42267   , -9.239022  ,  9.379146  ,  8.499881  ,
        -1.0592816 ,  3.343786  , -7.802612  , -0.59463346,  0.26447597,
         5.5073943 , -4.10698   ,  4.289078  , -2.8172052 ,  3.615014  ,
        -4.1612988 , -3.6209643 ,  6.21853   , -6.946047  , -1.0828308 ,
        -5.8267713 ,  2.2258759 , -3.8601205 , -1.6974078 , -5.313418  ,
         7.579579  ,  2.9187472 ,  8.540426  ,  1.5523202 ,  1.0841806 ],
       [-2.8941853 ,  4.4741874 , -4.4475656 ,  2.3820994 ,  1.7478833 ,
        -2.5046248 , -5.208329  , -1.6937687 , -8.134756  ,  2.6468298 ,
        -4.316364  ,  5.56554   , -5.732199  , -1.738495  , -9.344659  ,
         0.7084658 ,  4.4358387 , -2.9008987 , -4.9486394 ,  9.695301  ,
         8.366519  , -6.2474537 , -6.3494725 ,  1.9546974 ,  4.1576147 ,
        -9.167901  ,  4.6070666 ,  8.788586  ,  6.864423  ,  2.2319884 ],
       [-4.6657157 , -9.558953  ,  6.657227  ,  4.44013   ,  2.1730306 ,
         2.5904045 ,  0.5800097 ,  6.255036  , -8